In [1]:
import pandas as pd


In [44]:
train_data = pd.read_csv('../data/datas/train.csv')
test_data = pd.read_csv('../data/datas/test_nvPHrOx.csv')

In [118]:
sampleSubmission = pd.read_csv('../data/datas/sample_submission_poy1UIu.csv')

In [45]:
train_new_data = pd.read_csv('Title_data.csv',lineterminator='\n')

In [53]:
training_title = train_data.merge(train_new_data,left_on='Webpage_id',right_on='Webpage_id',how='inner')


In [55]:
testing_title = test_data.merge(train_new_data,left_on='Webpage_id',right_on='Webpage_id',how='inner')

## Put the label encoder tags

In [59]:
from sklearn.preprocessing import LabelEncoder 
training_le=LabelEncoder()
training_title['Tag'] = training_le.fit_transform(training_title['Tag'])

In [63]:
training_le.classes_

array(['clinicalTrials', 'conferences', 'forum', 'guidelines', 'news',
       'others', 'profile', 'publication', 'thesis'], dtype=object)

In [110]:
tagmap={}
for i,k in enumerate(training_le.classes_):
    tagmap.update({i:k})

'clinicalTrials'

In [111]:
def tagmap_function(idn):
    return tagmap[idn]

In [70]:
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import cross_val_score, cross_val_predict

from sklearn.metrics import roc_auc_score

from scipy.sparse import hstack

In [65]:
training_title.nunique()

Webpage_id    53447
Domain         3974
Url           53222
Tag               9
Title         43563
dtype: int64

In [67]:
training_title.Tag.unique()

array([4, 0, 1, 6, 2, 7, 8, 3, 5])

In [68]:
Tags = pd.get_dummies(training_title.Tag)
Tags.head()

,0,1,2,3,4,5,6,7,8
0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0


### Replace Nan as title : unkown

In [76]:
training_title.isnull().sum()

Webpage_id      0
Domain          0
Url             0
Tag             0
Title         220
dtype: int64

In [79]:
testing_title.isnull().sum()

Webpage_id      0
Domain          0
Url             0
Title         153
dtype: int64

In [77]:
training_title.fillna(value='Unknown class',inplace=True)

In [80]:
testing_title.fillna(value='Unknown class',inplace=True)

In [81]:
train_text = training_title['Title']
test_text = testing_title['Title']
all_text = pd.concat([train_text, test_text])


In [82]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 3),
    max_features=18000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [84]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 7),
    max_features=60000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [85]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

In [88]:
predictions = []
submission = pd.DataFrame.from_dict({'Webpage_id': testing_title['Webpage_id']})
for i in range(9):
    train_target = Tags[i].values
    classifier = LogisticRegression(C=2.65, solver='sag')

    #cv_prediction = cross_val_predict(classifier, train_features, train_target, cv=3, method='predict_proba')
    #predictions.append(cv_prediction[:, 1])
    #print('CV score for class {} is {}'.format(i, cv_score))

    classifier.fit(train_features, train_target)
    submission[str(i)] = classifier.predict_proba(test_features)[:, 1]

In [89]:
predictions_2 = []
submission_2 = pd.DataFrame.from_dict({'Webpage_id': testing_title['Webpage_id']})
for i in range(9):
    train_target = Tags[i].values
    classifier = LogisticRegression(C=2.6, solver='sag')

    #cv_prediction = cross_val_predict(classifier, train_features, train_target, cv=3, method='predict_proba')
    #predictions_2.append(cv_prediction[:, 1])
    #print('CV score for class {} is {}'.format(i, cv_score))

    classifier.fit(train_features, train_target)
    submission_2[str(i)] = classifier.predict_proba(test_features)[:, 1]

In [90]:
predictions_words = []
submission_words = pd.DataFrame.from_dict({'Webpage_id': testing_title['Webpage_id']})
for i in range(9):
    train_target = Tags[i].values
    classifier = LogisticRegression(C=2.6, solver='sag')

    #cv_prediction = cross_val_predict(classifier, train_word_features, train_target, cv=3, method='predict_proba')
    #predictions_words.append(cv_prediction[:, 1])
    #print('CV score for class {} is {}'.format(i, cv_score))

    classifier.fit(train_word_features, train_target)
    submission_words[str(i)] = classifier.predict_proba(test_word_features)[:, 1]

In [92]:
predictions_chars = []
submission_chars = pd.DataFrame.from_dict({'Webpage_id': testing_title['Webpage_id']})
for i in range(9):
    train_target = Tags[i].values
    classifier = LogisticRegression(C=2.6, solver='sag')

    #cv_prediction = cross_val_predict(classifier, train_char_features, train_target, cv=3, method='predict_proba')
    #predictions_chars.append(cv_prediction[:, 1])
    #print('CV score for class {} is {}'.format(i, cv_score))

    classifier.fit(train_char_features, train_target)
    submission_chars[str(i)] = classifier.predict_proba(test_char_features)[:, 1]

In [93]:
submission.head()

,Webpage_id,0,1,2,3,4,5,6,7,8
0,31,0.924659,0.017888,0.002548,0.001383,0.007473,0.016901,0.000459,0.011384,0.004350
1,32,0.575304,0.001088,0.000820,0.000879,0.018263,0.722397,0.000679,0.001757,0.000425
2,33,0.575304,0.001088,0.000820,0.000879,0.018263,0.722397,0.000679,0.001757,0.000425
3,34,0.575304,0.001088,0.000820,0.000879,0.018263,0.722397,0.000679,0.001757,0.000425
4,35,0.575304,0.001088,0.000820,0.000879,0.018263,0.722397,0.000679,0.001757,0.000425


In [94]:
submission_2.head()

,Webpage_id,0,1,2,3,4,5,6,7,8
0,31,0.923320,0.018007,0.002593,0.001409,0.007658,0.017325,0.000471,0.011577,0.004429
1,32,0.577468,0.001106,0.000841,0.000894,0.018232,0.720312,0.000696,0.001787,0.000434
2,33,0.577468,0.001106,0.000841,0.000894,0.018232,0.720312,0.000696,0.001787,0.000434
3,34,0.577468,0.001106,0.000841,0.000894,0.018232,0.720312,0.000696,0.001787,0.000434
4,35,0.577468,0.001106,0.000841,0.000894,0.018232,0.720312,0.000696,0.001787,0.000434


In [95]:
submission_chars.head()

,Webpage_id,0,1,2,3,4,5,6,7,8
0,31,0.902779,0.015111,0.004611,0.001752,0.022071,0.026143,0.000433,0.008950,0.006694
1,32,0.906564,0.001267,0.001078,0.001229,0.012091,0.426166,0.000840,0.001293,0.000505
2,33,0.906564,0.001267,0.001078,0.001229,0.012091,0.426166,0.000840,0.001293,0.000505
3,34,0.906564,0.001267,0.001078,0.001229,0.012091,0.426166,0.000840,0.001293,0.000505
4,35,0.906564,0.001267,0.001078,0.001229,0.012091,0.426166,0.000840,0.001293,0.000505


In [96]:
submission_words.head()

,Webpage_id,0,1,2,3,4,5,6,7,8
0,31,0.809939,0.034637,0.006563,0.003456,0.012600,0.067951,0.005238,0.047461,0.006851
1,32,0.157345,0.005226,0.004522,0.002675,0.025294,0.696751,0.003343,0.011056,0.002089
2,33,0.157345,0.005226,0.004522,0.002675,0.025294,0.696751,0.003343,0.011056,0.002089
3,34,0.157345,0.005226,0.004522,0.002675,0.025294,0.696751,0.003343,0.011056,0.002089
4,35,0.157345,0.005226,0.004522,0.002675,0.025294,0.696751,0.003343,0.011056,0.002089


In [99]:
submission[['0', '1', '2', '3', '4','5','6','7','8']].values

array([[9.24659400e-01, 1.78877547e-02, 2.54759353e-03, ...,
        4.59452847e-04, 1.13844400e-02, 4.35049474e-03],
       [5.75304178e-01, 1.08800192e-03, 8.20348756e-04, ...,
        6.79087590e-04, 1.75717750e-03, 4.25322509e-04],
       [5.75304178e-01, 1.08800192e-03, 8.20348756e-04, ...,
        6.79087590e-04, 1.75717750e-03, 4.25322509e-04],
       ...,
       [5.65696120e-04, 1.80163793e-03, 4.87833867e-03, ...,
        5.47940707e-01, 2.64641950e-03, 9.69269512e-04],
       [5.23827384e-04, 2.99033929e-03, 5.50304347e-03, ...,
        1.14694689e-01, 2.67934168e-03, 1.28921738e-03],
       [5.59408967e-04, 1.75270436e-03, 5.49673086e-03, ...,
        2.92896531e-01, 2.55543368e-03, 2.19089179e-03]])

In [100]:
blend_1 = 0.5*submission[['0', '1', '2', '3', '4','5','6','7','8']].values+0.5*submission_2[['0', '1', '2', '3', '4','5','6','7','8']].values
blend_2 = (0.35*submission[['0', '1', '2', '3', '4','5','6','7','8']].values+0.35*submission_2[['0', '1', '2', '3', '4','5','6','7','8']].values+
           0.15*submission_words[['0', '1', '2', '3', '4','5','6','7','8']].values+0.15*submission_chars[['0', '1', '2', '3', '4','5','6','7','8']].values)


In [101]:
predictions_1 = np.round(np.argmax(blend_1, axis=1)).astype(int)
predictions_2 = np.round(np.argmax(blend_2, axis=1)).astype(int)

In [120]:
sampleSubmission['Tag'] = predictions_1
sampleSubmission['Tag'] =sampleSubmission['Tag'].apply(tagmap_function)
sampleSubmission.to_csv("LR_Prediction_1.csv", index=False)

In [121]:
sampleSubmission['Tag'] = predictions_2
sampleSubmission['Tag'] =sampleSubmission['Tag'].apply(tagmap_function)
sampleSubmission.to_csv("LR_Prediction_2.csv", index=False)